# Exercício 1

Uma boa parte do trabalho das disciplinas de Física Experimental é o de calcular ajustes de reta para um dado conjunto de dados e uma teoria.
Considere um conjunto de dados $\{(x_1,y_1),(x_2,y_2),...,(x_n,y_n)\}$ para o qual a teoria diz que a relação entre $x$ e $y$ é dada por:

$$
y = a + bx
$$

Sabendo que as medidas estão sujeitas a várias formas de imprecisão e que a teoria não conta com todas tais contribuições, consideremos o modelo pára os dados

$$
y_i = a + bx_i + \xi_i
$$

onde $\xi_i$ são valores de ruído que assumimos ser Gausiano.

Os valores de $a$ e $b$ que melhor descrevem essa teoria devem ser aqueles que maximizam a verossimilhança de obsevar um certo valor de ruído, ou seja, minimizam $\sum_i\xi_i^2 = \sum_i(y_i - a - bx_i)^2$.

A solução desse problema é dada por:
$$
\hat{a} = \bar{y} - \hat{b}\bar{x}\\
\hat{b} = \frac{\sum_i(x_i-\bar{x})(y_i-\bar{y})}{\sum_i(x_i-\bar{x})^2}
$$

onde
$$
\bar{x} = \frac{1}{n}\sum_i x_i\\
\bar{y} = \frac{1}{n}\sum_i y_i
$$
são as médias amostrais de $x$ e $y$

Com seu conhecimento de Python, implemente funções para calcular a os coeficientes de ajuste da reta e aplique-os para o seguinte conjunto de dados:

In [1]:
from numpy.random import randn
a_real, b_real = 1.75, 3.5
amostra = {x: y for (x, y) in zip(range(-10,11), [a_real + b_real*x_i + randn() for x_i in range(-10,11)])}